In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hepatitis/GSE140249'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Distinct serum microRNA profiles in patients with active autoimmune hepatitis"
!Series_summary	"Serum microRNAs are useful for the early diagnosis and management of autoimmune hepatitis"
!Series_overall_design	"Serum microRNA profiles of patients with autoimmune hepatitis (AIH),  primary biliary cholangitis (PBC), overlap syndrome (OS), or without hepatic diseases."
Sample Characteristics Dictionary:
{0: ['Sex: F', 'Sex: M'], 1: ['age: 32', 'age: 61', 'age: 74', 'age: 27', 'age: 72', 'age: 28', 'age: 71', 'age: 76', 'age: 38', 'age: 41', 'age: 68', 'age: 51', 'age: 73', 'age: 69', 'age: 66', 'age: 63', 'age: 56', 'age: 45', 'age: 77', 'age: 59', 'age: 48', 'age: 49', 'age: 75', 'age: 50', 'age: 64', 'age: 58', 'age: 40', 'age: 52', 'age: 46', 'age: 62'], 2: ['disease state: AIH', 'disease state: PBC', 'disease state: OS', 'disease state: Healthy'], 3: ['treatment: Pre', 'treatment: Post', 'treatment: N/A'], 4: ['tissue: serum']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determining if gene expression data is available
is_gene_available = True  # Based on the output, it is likely gene expression data related to microRNA profiles

# Data availability and mapping keys
trait_row = 2  # disease state
age_row = 1    # age
gender_row = 0 # gender

# Converting trait (Hepatitis) data
def convert_trait(value):
    val = value.split(': ')[-1]
    if val == 'AIH':
        return 1
    elif val in ['PBC', 'OS', 'Healthy']:
        return 0
    else:
        return None

# Converting age data
def convert_age(value):
    try:
        return float(value.split(': ')[-1])
    except ValueError:
        return None

# Converting gender data
def convert_gender(value):
    val = value.split(': ')[-1]
    if val == 'F':
        return 0
    elif val == 'M':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE140249', './preprocessed/Hepatitis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hepatitis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Hepatitis/trait_data/GSE140249.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM4158202': [1.0, 32.0, 0.0], 'GSM4158203': [1.0, 61.0, 0.0], 'GSM4158204': [1.0, 74.0, 1.0], 'GSM4158205': [1.0, 27.0, 0.0], 'GSM4158206': [1.0, 72.0, 1.0], 'GSM4158207': [1.0, 28.0, 1.0], 'GSM4158208': [1.0, 71.0, 1.0], 'GSM4158209': [1.0, 76.0, 1.0], 'GSM4158210': [1.0, 38.0, 0.0], 'GSM4158211': [1.0, 41.0, 0.0], 'GSM4158212': [1.0, 68.0, 1.0], 'GSM4158213': [1.0, 51.0, 0.0], 'GSM4158214': [1.0, 73.0, 0.0], 'GSM4158215': [1.0, 69.0, 0.0], 'GSM4158216': [1.0, 68.0, 0.0], 'GSM4158217': [1.0, 72.0, 0.0], 'GSM4158218': [1.0, 66.0, 0.0], 'GSM4158219': [1.0, 66.0, 0.0], 'GSM4158220': [1.0, 68.0, 0.0], 'GSM4158221': [1.0, 63.0, 0.0], 'GSM4158222': [1.0, 56.0, 0.0], 'GSM4158223': [1.0, 69.0, 0.0], 'GSM4158224': [1.0, 45.0, 0.0], 'GSM4158225': [1.0, 77.0, 0.0], 'GSM4158226': [1.0, 59.0, 0.0], 'GSM4158227': [1.0, 48.0, 0.0], 'GSM4158228': [1.0, 49.0, 0.0], 'GSM4158229': [1.0, 72.0, 0.0], 'GSM4158230': [1.0, 32.0, 0.0], 'GSM4158231': [1.0, 74.0, 1.0], 'GSM4158232': [1.0, 27.0, 0.0], 'GSM415

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065',
       'MIMAT0000066', 'MIMAT0000067', 'MIMAT0000068', 'MIMAT0000069',
       'MIMAT0000070', 'MIMAT0000071', 'MIMAT0000072', 'MIMAT0000073',
       'MIMAT0000074', 'MIMAT0000075', 'MIMAT0000076', 'MIMAT0000077',
       'MIMAT0000078', 'MIMAT0000079', 'MIMAT0000080', 'MIMAT0000081'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066'], 'miRNA': ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066'], 'miRNA_ID_LIST': ['hsa-let-7a-5p', 'hsa-let-7b-5p', 'hsa-let-7c-5p', 'hsa-let-7d-5p', 'hsa-let-7e-5p']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which key stores the identifiers and which key stores the gene symbols
identifier_key = 'ID'
gene_symbol_key = 'miRNA_ID_LIST'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hepatitis/gene_data/GSE140249.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hepatitis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE140249', './preprocessed/Hepatitis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Hepatitis/GSE140249.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
